In [ ]:
!pip install requests beautifulsoup4




In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import json

def get_all_links_json(url, domain, crawled_links=None):
    if crawled_links is None:
        crawled_links = set()

    urls_found = []

    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
    except Exception as e:
        print(f"Failed to retrieve {url}: {e}")
        return list(crawled_links)

    soup = BeautifulSoup(response.text, 'html.parser')
    for a_tag in soup.find_all('a', href=True):
        href = a_tag['href']
        full_url = urljoin(url, href)
        parsed_url = urlparse(full_url)

        if domain in parsed_url.netloc:
            clean_url = full_url.split('#')[0]
            if clean_url not in crawled_links:
                crawled_links.add(clean_url)
                urls_found.append(clean_url)
                print(f"Discovered: {clean_url}")
                # Recursive crawl
                get_all_links_json(clean_url, domain, crawled_links)

    return list(crawled_links)


base_url = "https://kcoa-africa.org/the-smart-farmers-guide-to-agricultural-loans-borrowing-without-regret/"
domain = "kcoa-africa.org"

all_internal_links = get_all_links_json(base_url, domain)
result_json = json.dumps({
    "base_url": base_url,
    "total_links": len(all_internal_links),
    "internal_links": all_internal_links
}, indent=2)


print(result_json)




Discovered: https://kcoa-africa.org/the-smart-farmers-guide-to-agricultural-loans-borrowing-without-regret/
